# The 3 Common Synthetic Baseline Datasets

In [1]:
%load_ext autoreload
%autoreload 2

from synthetic_dataset_1 import make_ds1
from synthetic_dataset_2 import make_ds2
from synthetic_dataset_3 import make_ds3
from deepcqr_synth import \
    create_fit_predict_evaluate, \
    create_single_unit_model, \
    train_validation_test_random_splitter, \
    create_initializer_near_0_or_1, \
    get_range_of_fit_seeds, \
    evaluation_measures

import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
import tensorflow_probability as tfp

import pickle
import os
from glob import glob
from functools import partial

# Deep Tobit Type I

In [2]:
def tobit_type1_nll_tensorflow(y_true, y_pred, noise_sigma, lower_threshold):
    def nll_not_censored(y_true, y_pred, gamma):
        nrm = tfp.distributions.Normal(loc=0, scale=1)
        cens_labels = y_true <= 0
        return -tf.math.reduce_sum(
            np.log(gamma) + nrm.log_prob(
                gamma * tf.boolean_mask(y_true, ~cens_labels) - \
                tf.boolean_mask(y_pred, ~cens_labels)))
    
    def nll_censored(y_pred, gamma, lower_threshold):
        nrm = tfp.distributions.Normal(loc=0, scale=1)
        cens_labels = y_true <= 0
        return -tf.math.reduce_sum(
            nrm.log_cdf(gamma * lower_threshold - \
                        tf.boolean_mask(y_pred, cens_labels)))
    
    return \
        nll_not_censored(y_pred=y_pred, 
                         y_true=y_true, 
                         gamma=1 / noise_sigma) + \
        nll_censored(y_pred=y_pred,
                     gamma=1 / noise_sigma,
                     lower_threshold=lower_threshold)

In [3]:
# ---------------- 1st Synth DS ------------------
create_fit_predict_evaluate(
    splitter=partial(train_validation_test_random_splitter, seed=0, percent_test=0.33, percent_val_from_train=0.2),
    initalization_creator=create_initializer_near_0_or_1,
    fit_seeds=get_range_of_fit_seeds(),
    model_creator=partial(
        create_single_unit_model, 
        loss=lambda y_true, y_pred: tobit_type1_nll_tensorflow(
            noise_sigma=1, 
            y_true=y_true, 
            y_pred=y_pred, 
            lower_threshold=0),
        kernel_initializer=lambda seed: tf.keras.initializers.random_normal(mean=1.0, stddev=0.05, seed=seed),
        activation='linear',
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.1, clipnorm=1),
        num_x_features=3),
    verbose=False,
    dataset=make_ds1(verbose=True, num_samples=1000, seed=42)
)['evaluation']

fit seeds:   0%|          | 0/20 [00:00<?, ?it/s]

Censored observations: 274 of 1000 (0.27%)
Fraction censored of test: 0.2878787878787879


fit seeds:   5%|▌         | 1/20 [00:01<00:36,  1.94s/it]

Restoring model weights from the end of the best epoch.
Epoch 00020: early stopping
Current loss = 159.34573364257812, best loss before current = None


fit seeds:  10%|█         | 2/20 [00:03<00:33,  1.85s/it]

...

fit seeds:  95%|█████████▌| 19/20 [00:31<00:01,  1.61s/it]

Restoring model weights from the end of the best epoch.
Epoch 00014: early stopping
Current loss = 159.5835723876953, best loss before current = 159.221923828125


fit seeds: 100%|██████████| 20/20 [00:33<00:00,  1.69s/it]

Restoring model weights from the end of the best epoch.
Epoch 00026: early stopping
Current loss = 159.23318481445312, best loss before current = 159.221923828125
Best model weights: [array([[1.0300485 ],
       [1.0182496 ],
       [0.94238794]], dtype=float32)]
y_true > 0 = 235 of 330 (0.71)


{'all': [('r2', '0.648'), ('mae', '0.745'), ('rmse', '0.948')],
 'only_non_censored': [('r2', '0.381'), ('mae', '0.704'), ('rmse', '0.909')]}

In [4]:
# ---------------- 2nd Synth DS ------------------
create_fit_predict_evaluate(
    splitter=partial(train_validation_test_random_splitter, seed=0, percent_test=0.33, percent_val_from_train=0.2),
    initalization_creator=create_initializer_near_0_or_1,
    fit_seeds=get_range_of_fit_seeds(),
    model_creator=partial(
        create_single_unit_model, 
        loss=lambda y_true, y_pred: tobit_type1_nll_tensorflow(
            noise_sigma=1, 
            y_true=y_true, 
            y_pred=y_pred, 
            lower_threshold=0),
        kernel_initializer=lambda seed: tf.keras.initializers.random_normal(mean=1.0, stddev=0.05, seed=seed),
        activation='linear',
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.1, clipnorm=1),
        num_x_features=3),
    verbose=False,
    dataset=make_ds2(verbose=True, num_samples=1000, seed=42)
)['evaluation']

fit seeds:   0%|          | 0/20 [00:00<?, ?it/s]

Censored observations: 309 of 1000 (0.31%)
Fraction censored of test: 0.3181818181818182


fit seeds:   5%|▌         | 1/20 [00:01<00:31,  1.66s/it]

Restoring model weights from the end of the best epoch.
Epoch 00030: early stopping
Current loss = 211.37237548828125, best loss before current = None


fit seeds:  10%|█         | 2/20 [00:03<00:29,  1.63s/it]

...

fit seeds:  95%|█████████▌| 19/20 [00:32<00:01,  1.74s/it]

Restoring model weights from the end of the best epoch.
Epoch 00014: early stopping
Current loss = 213.30300903320312, best loss before current = 211.13604736328125


fit seeds: 100%|██████████| 20/20 [00:33<00:00,  1.68s/it]

Restoring model weights from the end of the best epoch.
Epoch 00011: early stopping
Current loss = 212.22894287109375, best loss before current = 211.13604736328125
Best model weights: [array([[0.96875584],
       [1.1288785 ],
       [1.1875395 ]], dtype=float32)]
y_true > 0 = 225 of 330 (0.68)


{'all': [('r2', '0.461'), ('mae', '0.911'), ('rmse', '1.359')],
 'only_non_censored': [('r2', '0.093'), ('mae', '0.954'), ('rmse', '1.394')]}

In [5]:
# ---------------- 3rd Synth DS ------------------
create_fit_predict_evaluate(
    splitter=partial(train_validation_test_random_splitter, seed=0, percent_test=0.33, percent_val_from_train=0.2),
    initalization_creator=create_initializer_near_0_or_1,
    fit_seeds=get_range_of_fit_seeds(),
    model_creator=partial(
        create_single_unit_model, 
        loss=lambda y_true, y_pred: tobit_type1_nll_tensorflow(
            noise_sigma=1, 
            y_true=y_true, 
            y_pred=y_pred, 
            lower_threshold=0),
        kernel_initializer=lambda seed: tf.keras.initializers.random_normal(mean=1.0, stddev=0.05, seed=seed),
        activation='linear',
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.1, clipnorm=1),
        num_x_features=3),
    verbose=False,
    dataset=make_ds3(verbose=True, num_samples=1000, seed=42)
)['evaluation']

fit seeds:   0%|          | 0/20 [00:00<?, ?it/s]

Censored observations: 264 of 1000 (0.26%)
Fraction censored of test: 0.2818181818181818


fit seeds:   5%|▌         | 1/20 [00:01<00:29,  1.57s/it]

Restoring model weights from the end of the best epoch.
Epoch 00020: early stopping
Current loss = 148.5384063720703, best loss before current = None


fit seeds:  10%|█         | 2/20 [00:03<00:28,  1.57s/it]

...

fit seeds:  95%|█████████▌| 19/20 [00:33<00:01,  1.79s/it]

Restoring model weights from the end of the best epoch.
Epoch 00014: early stopping
Current loss = 148.72471618652344, best loss before current = 147.27532958984375


fit seeds: 100%|██████████| 20/20 [00:34<00:00,  1.75s/it]

Restoring model weights from the end of the best epoch.
Epoch 00020: early stopping
Current loss = 147.6136016845703, best loss before current = 147.27532958984375
Best model weights: [array([[0.96592826],
       [1.1058626 ],
       [0.9715684 ]], dtype=float32)]
y_true > 0 = 237 of 330 (0.72)


{'all': [('r2', '0.663'), ('mae', '0.719'), ('rmse', '0.906')],
 'only_non_censored': [('r2', '0.352'), ('mae', '0.718'), ('rmse', '0.901')]}

# Mean by Expected Value via QR with $\theta=0.05, 0.10, \dots, 0.95$

We run on HPC and re-aggregate the results here.

In [6]:
def analyze_results_from_hpc(thetas_dir):
    results = dict([pickle.load(open(f, 'rb')) for f in glob(os.path.join(thetas_dir, 'theta*.pkl'))])
    ystar_true = np.array(results[0.05]['ystar_true'])
    ystar_pred = 0.05 * sum(np.array(results[round(theta, 2)]['ystar_pred']) 
                            for theta in np.arange(0.05, 0.96, 0.05))
    return {
        'ystar_pred': ystar_pred,
        'ystar_true': ystar_true, 
        'evaluation': evaluation_measures(y_pred=ystar_pred, y_true=ystar_true)
    }

In [7]:
analyze_results_from_hpc('../data/thetas/synth_ds_1/')['evaluation']

y_true > 0 = 235 of 330 (0.71)


{'all': [('r2', '0.649'), ('mae', '0.750'), ('rmse', '0.946')],
 'only_non_censored': [('r2', '0.410'), ('mae', '0.690'), ('rmse', '0.888')]}

In [8]:
analyze_results_from_hpc('../data/thetas/synth_ds_2/')['evaluation']

y_true > 0 = 225 of 330 (0.68)


{'all': [('r2', '0.485'), ('mae', '0.881'), ('rmse', '1.328')],
 'only_non_censored': [('r2', '0.203'), ('mae', '0.860'), ('rmse', '1.307')]}

In [9]:
analyze_results_from_hpc('../data/thetas/synth_ds_3/')['evaluation']

y_true > 0 = 237 of 330 (0.72)


{'all': [('r2', '0.669'), ('mae', '0.711'), ('rmse', '0.898')],
 'only_non_censored': [('r2', '0.401'), ('mae', '0.681'), ('rmse', '0.866')]}

==> Again, better prediction quality by ensemble QR than by Tobit, particularly for the heteroskedastic dataset.